In [ ]:
import signac

project = signac.get_project()

In [ ]:
import sys
from ipywidgets import interact

JOB = None

schema = project.detect_schema()
ns = schema['n'][int]
betaPs = schema['betaP'][float]
seeds = schema['seed'][int]

@interact(n=ns, betaP=betaPs, seed=seeds)
def select(n=5, betaP=13.2, seed=0):
    global JOB
    jobs = project.find_jobs(dict(n=n, betaP=betaP, seed=seed))
    if len(jobs) == 0:
        print("No jobs found for this selection.", file=sys.stderr)
    elif len(jobs) > 1:
        print("Multiple jobs matched this selection.", file=sys.stderr)
    else:
        JOB = list(jobs)[0]
        print("Selected", JOB)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import gsd.hoomd
%matplotlib inline

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    N = traj[-1].particles.N
# alternately, use a custom log operation to log phi directly

log = np.genfromtxt(fname=JOB.fn('log.dat'), names=True)
psi = np.load(JOB.fn('order.npz'))

fig, ax = plt.subplots(figsize=(4, 2.2), dpi=140)
ax2 = ax.twinx()

ax.plot(log['timestep'], N * JOB.doc.poly_area / log['volume'])
ax.set_xlabel('time step')
ax.set_ylabel('Packing Fraction')

ax2.plot(psi['steps'], np.absolute(psi['psi']).mean(axis=1), color='red')
ax2.set_ylabel('Hexatic Order Parameter ($\psi$)', color='red')

plt.show()

In [ ]:
from matplotlib import collections, patches, cm, colors
import freud
import geometry

def quat2ang(quats):
    """Convert quaternions to angles."""
    return np.mod(2*np.arctan2(quats[:, 3], quats[:, 0]), 2*np.pi)

def draw_config(fig, ax, box, pos, psi, nverts):
    ax.set_xlim((-box.Lx/2, box.Lx/2))
    ax.set_ylim((-box.Ly/2, box.Ly/2))
    verts = geometry.get_vertices(nverts)

    cmap = cm.get_cmap('viridis')

    for i, p in enumerate(pos):
        ax.add_patch(patches.Polygon(xy=p[:2]+verts, color=cmap(np.absolute(psi[i]))))
    
def draw_pmft(fig, ax, pmft, nverts):
    im = ax.contourf(pmft.X, pmft.Y, pmft.PMFT)
    cb = fig.colorbar(im, ax=ax)
    cb.set_label("$k_b T$", fontsize=12)
    ax.add_patch(patches.Polygon(xy=geometry.get_vertices(nverts)))

def draw_voronoi(fig, ax, box, cells):
    """Draw voronoi tesselation"""
    cmap = cm.get_cmap('viridis')
    bounds = list(range(3, 9))
    cols = [cmap((i-bounds[0])/len(bounds)) for i in bounds]
    cmap = colors.LinearSegmentedColormap.from_list("my_cmap", cols, N=7, gamma=1.0)
    norm = colors.BoundaryNorm(bounds, cmap.N)
    
    cols = [len(cell) for cell in cells]
    patches = [plt.Polygon(cell[:, :2]) for cell in cells]
    patch_collection = collections.PatchCollection(patches, edgecolors='black')
    patch_collection.set_array(np.array(cols))
    patch_collection.set_cmap(cmap)
    
    ax.add_collection(patch_collection)
    ax.set_xlim((-box.Lx/2, box.Lx/2))
    ax.set_ylim((-box.Ly/2, box.Ly/2))
    cb = fig.colorbar(patch_collection, ax=ax, cmap=cmap, norm=norm, ticks=bounds, boundaries=bounds)

    cb.set_label("Number of sides", fontsize=12)
    
with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    num_frames = len(traj)-1 

In [ ]:
@interact(num=(0, num_frames))
def frame_demo(num=1):
    fix, ax = plt.subplots(1, 1, figsize=(8, 8))
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[num]
        
        box = freud.box.Box.from_box(frame.configuration.box[:2])
        hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
        hop.compute(box, frame.particles.position)
        
        draw_config(fig, ax, box, frame.particles.position, hop.psi, JOB.sp.n)

In [ ]:
@interact(num=(0, num_frames))
def pmft_demo(num=1):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[num]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2])
        pmft = freud.pmft.PMFTXY2D(4, 4, 300, 300)
        pmft.compute(box, frame.particles.position, quat2ang(frame.particles.orientation))

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    draw_pmft(fig, ax, pmft, JOB.sp.n)

In [ ]:
@interact(num=(0, num_frames))
def rdf_demo(num=1):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[num]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        rdf = freud.density.RDF(np.sqrt(box.Lx**2 + box.Ly**2)/5, box.Lx/1000)
        rdf.compute(box, frame.particles.position)

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.plot(rdf.R, rdf.RDF)

In [ ]:
@interact(num=(0, num_frames))
def rdf_demo(num=1):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[num]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        voronoi = freud.voronoi.Voronoi(box, np.sqrt(box.Lx**2 + box.Ly**2)/5)
        voronoi.compute(frame.particles.position, box)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    draw_voronoi(fig, ax, box, voronoi.polytopes)

In [ ]:
@interact(n=project.detect_schema()['n'][int], num_frames=(1, 10))
def transition_demo(n, num_frames=5):
    avg_hop = {}
    for betaP, group in project.find_jobs({"n": n}).groupby('betaP'):
        hops = []
        for job in group:
            with gsd.hoomd.open(job.fn('trajectory.gsd')) as traj:
                for frame in traj[-num_frames:]:
                    box = freud.box.Box.from_box(frame.configuration.box[:2])
                    hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
                    hop.compute(box, frame.particles.position)
                    hops.append(np.mean(np.absolute(hop.psi)))
        avg_hop[betaP] = np.mean(hops)
    fig, ax = plt.subplots(1, 1)
    ax.plot(avg_hop.keys(), avg_hop.values())